In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from subprocess import check_output

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Reading The Data

In [ ]:
# Read training and test data files
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape) #Printing the shape of the dataset 
print(test.shape) #printing the shape of test dataset

In [ ]:
#Printing the dimension of train_x and train_y
train_y = train['label'].astype('int32')
train_x = train.drop(['label'],axis = 1).astype('int32')
test_x = test.astype('int32')

#Printing the shape of train_x and train_y
train_x.shape, train_y.shape, test_x.shape

In [ ]:
#Reshaping the image 
train_x = train_x.values.reshape(-1,28,28,1)
#normalisation
train_x = train_x/255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x=test_x/255.0

#checking the updated shape 
train_x.shape, test_x.shape

In [ ]:
plt.imshow(test_x[900].reshape(28,28),cmap = matplotlib.cm.binary)
plt.show()

In [ ]:
fig = plt.figure(figsize = (8, 8))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_x[i])
plt.show()

In [ ]:
#convert label into categorical 
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape  #Printing the shape 

In [ ]:
#Printing the Labels for top 5 rows
print(train['label'].head())  #checking Data type 

In [ ]:
#Printing Categorical label
print(train_y[0:5,:])

# Defining Model

In [ ]:
#Defining the Model
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(28,28,1)),
      tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
      tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
      tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(256,activation='relu'),
      tf.keras.layers.Dropout(0.50),
      tf.keras.layers.Dense(10, activation='softmax')
                         
])
model.summary()

In [ ]:
#Defining the callback function 

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.999):
      print("\nReached 99.9% accuracy!")
      self.model.stop_training = True

callbacks = myCallback() 

In [ ]:
#Compiling and model training
#BatchSize = 50, epochs = 20 and optimizer = adam
Optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.0005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size =50, epochs=20, callbacks=[callbacks])

In [ ]:
results = model.predict(test_x)

#index with the maximum probability
results = np.argmax(results,axis=1)
results = pd.Series(results,name="Label")

In [ ]:
results

In [ ]:
submission = pd.concat([pd.Series(range(1,28001), name="ImageId"),results],axis=1)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)